In [ ]:
import requests
import json
from pprint import pprint

# Configuration
BASE_URL = "http://localhost:8182"

def pretty_print(data):
    """Pretty print JSON data"""
    print(json.dumps(data, indent=2))

print("✅ Setup complete!")

## Step 1: Discovery - What Can This Merchant Do?

Every UCP merchant publishes their capabilities at `/.well-known/ucp`. This tells agents:
- What services they support (shopping, payments, etc.)
- Which capabilities are available (checkout, fulfillment, discounts)
- Payment handlers and configurations

In [ ]:
# Discover merchant capabilities
response = requests.get(f"{BASE_URL}/.well-known/ucp")
discovery = response.json()

print("🔍 Merchant Discovery Profile:")
print("\nUCP Version:", discovery['ucp']['version'])
print("\nAvailable Services:")
for service_name, service_info in discovery['ucp']['services'].items():
    print(f"  - {service_name}: {service_info['version']}")

print("\nSupported Capabilities:")
for cap in discovery['ucp']['capabilities']:
    print(f"  - {cap['name']}")

# Store for later use
print("\n📄 Full Discovery Profile:")
pretty_print(discovery)

## Step 2: Create a Checkout Session

Before adding items, we need to create a checkout session. This is like opening a shopping cart.

In [ ]:
# Create checkout session
response = requests.post(
    f"{BASE_URL}/shopping/checkout",
    json={}
)
checkout = response.json()
checkout_id = checkout['id']

print(f"🛒 Created checkout session: {checkout_id}")
print(f"\nStatus: {checkout['status']}")
print(f"Currency: {checkout['currency']}")

print("\n📄 Full Checkout Response:")
pretty_print(checkout)

## Step 3: Add Items to Cart

Now let's add some products to our checkout. We'll add:
- 2 Rose Bouquets
- 1 Ceramic Pot

In [ ]:
# Add items to checkout
response = requests.post(
    f"{BASE_URL}/shopping/checkout/{checkout_id}",
    json={
        "items": [
            {
                "action": "add",
                "quantity": 2,
                "item": {"id": "roses_dozen"}
            },
            {
                "action": "add",
                "quantity": 1,
                "item": {"id": "pot_ceramic"}
            }
        ]
    }
)
checkout = response.json()

print("📦 Items added to cart!\n")
print("Line Items:")
for line_item in checkout['line_items']:
    item = line_item['item']
    print(f"  - {item['title']}: ${item['price']/100:.2f} x {line_item['quantity']}")

print("\nTotals:")
for total in checkout['totals']:
    amount = total['amount'] / 100
    print(f"  {total['type']}: ${amount:.2f}")

print(f"\nCheckout Status: {checkout['status']}")

## Step 4: Add Buyer Information

To proceed, we need to provide buyer details like name and email.

In [ ]:
# Update buyer information
response = requests.post(
    f"{BASE_URL}/shopping/checkout/{checkout_id}",
    json={
        "buyer": {
            "full_name": "Demo User",
            "email": "demo@example.com"
        }
    }
)
checkout = response.json()

print("👤 Buyer information added")
print(f"Name: {checkout['buyer']['full_name']}")
print(f"Email: {checkout['buyer']['email']}")
print(f"\nCheckout Status: {checkout['status']}")

## Step 5: Add Shipping Address

For physical goods, we need a shipping address.

In [ ]:
# Add shipping information via fulfillment
response = requests.post(
    f"{BASE_URL}/shopping/checkout/{checkout_id}",
    json={
        "fulfillment": {
            "methods": [
                {
                    "type": "shipping",
                    "destinations": [
                        {
                            "address": {
                                "street_address": "123 Main St",
                                "address_locality": "Amsterdam",
                                "postal_code": "1012 AB",
                                "address_country": "NL"
                            }
                        }
                    ]
                }
            ]
        }
    }
)
checkout = response.json()

print("📍 Shipping address added")
if 'fulfillment' in checkout:
    fulfillment = checkout['fulfillment']
    if 'methods' in fulfillment:
        for method in fulfillment['methods']:
            print(f"\nMethod: {method['type']}")
            if 'available_options' in method:
                print("\nAvailable shipping options:")
                for option in method['available_options']:
                    print(f"  - {option['title']}: ${option['total']/100:.2f}")

print(f"\nCheckout Status: {checkout['status']}")

## Step 6: Select Shipping Option

Choose one of the available shipping methods.

In [ ]:
# Select a shipping option (use first available)
shipping_method = checkout['fulfillment']['methods'][0]
shipping_option_id = shipping_method['available_options'][0]['id']

response = requests.post(
    f"{BASE_URL}/shopping/checkout/{checkout_id}",
    json={
        "fulfillment": {
            "methods": [
                {
                    "type": "shipping",
                    "groups": [
                        {"selected_option_id": shipping_option_id}
                    ]
                }
            ]
        }
    }
)
checkout = response.json()

print("✅ Shipping option selected")
print("\nUpdated totals:")
for total in checkout['totals']:
    amount = total['amount'] / 100
    print(f"  {total['type']}: ${amount:.2f}")

print(f"\nCheckout Status: {checkout['status']}")

## Step 7: Complete the Order

Finally, complete the checkout with payment information.

In [ ]:
# Complete checkout with mock payment
response = requests.post(
    f"{BASE_URL}/shopping/checkout/{checkout_id}/complete",
    json={
        "payment": {
            "instrument": {
                "type": "card",
                "credential": {
                    "type": "token",
                    "token": "success_token"  # Mock token that always succeeds
                }
            }
        }
    }
)
order = response.json()

print("🎉 Order completed successfully!")
print(f"\nOrder ID: {order['id']}")
print(f"Status: {order['status']}")

print("\nOrder Summary:")
for line_item in order['line_items']:
    item = line_item['item']
    print(f"  - {item['title']}: ${item['price']/100:.2f} x {line_item['quantity']}")

print("\nFinal Totals:")
for total in order['totals']:
    amount = total['amount'] / 100
    print(f"  {total['type']}: ${amount:.2f}")

if 'confirmation' in order:
    conf = order['confirmation']
    print(f"\nConfirmation: {conf['title']}")
    if 'message' in conf:
        print(f"Message: {conf['message']}")

## 🎯 Experiment: Try Different Scenarios

Now that you've seen the basic flow, try modifying the cells above to:

1. **Add different products** - Change the item IDs in Step 3
2. **Use a failing payment** - Change `success_token` to `fail_token` in Step 7
3. **Add more items** - Increase quantities or add more line items
4. **Change shipping address** - Try different countries or postal codes

### Available Product IDs
- `roses_dozen` - Dozen Roses ($35)
- `pot_ceramic` - Ceramic Pot ($15)
- `tulips_bunch` - Tulip Bunch ($25)
- `vase_glass` - Glass Vase ($20)

### Payment Tokens
- `success_token` - Payment succeeds
- `fail_token` - Payment fails

## Next Steps

- Explore the [UCP Specification](https://ucp.dev)
- Check out the [A2A Demo](../a2a/) for AI-powered shopping
- Integrate UCP into your own applications
- Try the Node.js implementation in `rest/nodejs/`